#### imports

In [ ]:
import csv
import requests
import math
import pprint
import os
import json
import pandas as pd
from pandas.io.json import json_normalize
from datetime import datetime, date, time
from IPython.display import clear_output

In [ ]:
cwd = os.getcwd()
print cwd

In [ ]:
pp = pprint.PrettyPrinter(indent=2)

#### common functions

In [ ]:
def findFromPathString(path_string, JSON, separator="/", debug=False):     
    paths_ = path_string.split(separator)
    data = JSON
    if debug :
        print "\n data : "
        print pp.pprint(data)
    try : 
        for i in range(0,len(paths_)):
            data = data[paths_[i]]
        return data
    except :
        return False

In [ ]:
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

#### settings

In [ ]:

root_url_corpos = "http://opencorporatefacts.fr/api/corporates"
# params_corpos = { 'page' : 1 , 'id' : None , 'CompanyNumber' : None }
results_path_corpos = "hydra:member"
output_filename_corpos = "datasets/corporations/corpos"
headers_corpos = {'accept': 'application/ld+json'}

root_url_accounts = "http://opencorporatefacts.fr/api/compte_de_resultats"
# params_accounts = { 'page' : 1, 'Corporate' : None }
results_path_accounts = "hydra:member"
output_filename_accounts = "datasets/account_results/accounts"
headers_accounts = {'accept': 'application/ld+json'}

root_url_companies = "http://api.cquest.org/company/" ### pass CompanyNumber after root_url
output_filename_companies = "datasets/companies_data/companies"


#### tests

In [ ]:
#print pp.pprint(results)

#### main requests functions

In [ ]:
def generate_filename( output_filename, page_treshold, split_files_treshold_pages, max_page, only_filename=False ) :
    
    start_page = page_treshold
    end_page = page_treshold + split_files_treshold_pages - 1
    if end_page > max_page : 
        end_page = max_page
    
    if only_filename : 
        prefix = ''
    else : 
        prefix = cwd + '/'
        
    filename = prefix + output_filename + '-pageStart_' + str(start_page) + '_to_' + str(end_page)
    
    return filename
    
def get_req_results ( 
    url, 
    pages_range, 
    number_of_pages, 
    results_path, 
    output_filename, 
    start_page=1, 
    split_files_treshold_pages=1000,
    debug=False, 
    out_formats=["csv"] 
    ) : 
    ''' 
    retrieve results from requests 
    and write csv 
    works with API platform only for now
    '''
    
    print "url : {}\n".format(url)
    
    start_time = datetime.now()
    
    ### count pages
    count = 0
    page_treshold = 0
    
    ### requests and writing file
    
    for page in pages_range : 
    
        if page > 0 and page >= start_page  : 
            
            ### treshold and split into several files
            if page % split_files_treshold_pages == 0 :
                page_treshold = page
                count = 0
                
            ### output files
            if "csv" in out_formats :
                file_data_csv_filename  = generate_filename( output_filename, page_treshold, split_files_treshold_pages, number_of_pages )
                file_data_csv  = file_data_csv_filename + '.csv'

            if "json" in out_formats : 
                file_data_json_filename = generate_filename( output_filename, page_treshold, split_files_treshold_pages, number_of_pages )
                file_data_json = file_data_json_filename + '.json'
            
            if debug == False : 
                clear_output( wait=True ) 
            
            resp = requests.get( url, params = { 'page' : page } ).json()
            results = findFromPathString( results_path, resp, debug=debug )
            
            page_time = datetime.now()
            delta_time = page_time - start_time
            
            filename = generate_filename( output_filename, page_treshold, split_files_treshold_pages, number_of_pages, only_filename=True )
            
            print "\nPage number : {} / {} \
                    \ndelta_time (after request) : {}\
                    \npage_treshold : {}\
                    \nfilename : {}\
                    \n".format(page, number_of_pages, delta_time, page_treshold, filename)
            
            if results : 
                # loop results
                for data in results : 

                    if debug : 
                        # print pp.pprint(data) 
                        print data
                        print
                    data_flat = flatten_json(data)
                    # print data_flat

                    ### CSV
                    if "csv" in out_formats : 

                        ### creates headers
                        if count == 0:
                            with open(file_data_csv, 'w') as csv_file :
                                csv_writer = csv.writer(csv_file, delimiter=';')
                                header = data.keys()
                                csv_writer.writerow(header)
                                count += 1
                        else : 
                            with open(file_data_csv, 'a') as csv_file :
                                csv_writer = csv.writer(csv_file, delimiter=';')
                                csv_writer.writerow( data.values() )
                                count += 1


                    ### JSON
                    if "json" in out_formats : 
                        if count == 0 :
                            with open(file_data_json, 'w') as json_file :
                                json_file.write(json.dumps( [data] ).encode())         #If empty, write an array
                                count += 1
                        else : 
                            with open(file_data_json, 'ab+') as json_file :
                                json_file.seek(-1,2)           
                                json_file.truncate()                           #Remove the last character, open the array
                                json_file.write(' , '.encode())                #Write the separator
                                json_file.write(json.dumps(data).encode())     #Dump the dictionary
                                json_file.write(']'.encode())                  #Close the array
                                count += 1


#### getting data

In [ ]:
### TEST / get accounts 
number_of_pages_accounts_test = 30
pages_range_test = range(number_of_pages_accounts_test)
get_req_results( 
    root_url_accounts, 
    pages_range_test, 
    number_of_pages_accounts_test,
    results_path_accounts, 
    output_filename_accounts+'_test', 
    start_page=1, 
    split_files_treshold_pages=10,
    debug=True, 
    out_formats=["json"] 
) 

In [ ]:
### get corpos data
number_of_results_corpos = 577297
results_per_page = 30
number_of_pages_corpos = int( math.ceil( number_of_results_corpos / results_per_page ) )
print number_of_pages_corpos
pages_to_scrap_corpos = range( number_of_pages_corpos + 1 )
print pages_to_scrap_corpos[:3]

start_page_corpos = 1
file_suffix_corpos = '_06'
file_treshold_corpos = 2500

get_req_results( 
    root_url_corpos, 
    pages_to_scrap_corpos, 
    number_of_pages_corpos,
    results_path_corpos, 
    output_filename_corpos+file_suffix_corpos, 
    start_page=start_page_corpos, 
    split_files_treshold_pages=file_treshold_corpos,
    debug=False, 
    out_formats=["json"] 
) 

In [ ]:
### get accounts data
number_of_results_account = 316182
results_per_page = 30
number_of_pages_accounts = int( math.ceil( number_of_results_account / results_per_page ) )
print number_of_pages_accounts
pages_to_scrap_accounts = range( number_of_pages_accounts + 1 )
print pages_to_scrap_accounts[:3]

start_page_accounts = 1
file_suffix_accounts = '_06'
file_treshold_accounts = 500

get_req_results( 
    root_url_accounts, 
    pages_to_scrap_accounts, 
    number_of_pages_accounts,
    results_path_accounts, 
    output_filename_accounts+file_suffix_accounts, 
    start_page=start_page_accounts, 
    split_files_treshold_pages=file_treshold_accounts,
    debug=False, 
    out_formats=["json"] 
) 